In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json

In [2]:
import matplotlib as mpl
mpl.use("pgf")
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "font.serif": "Times New Roman"
})
plt.rcParams['text.usetex'] = True

In [3]:
traces_to_plot = {
    "RPM": "../downsampled_data_10000_rows_first_arrivals.csv___lshare_fair___50___2000___avg.jsonl",
    "VTC": "../downsampled_data_10000_rows_first_arrivals.csv___vtc_fair___2000___avg.jsonl",
    "FairServe": "../downsampled_data_10000_rows_first_arrivals.csv___fs_fair_interaction_limit_expect___50___2000___avg.jsonl",
    "FairServe (WSC)": "../downsampled_data_10000_rows_first_arrivals.csv___fs_fair_wsc_expect___50___2000___avg.jsonl"
}

In [8]:
cutoff_time = 100

In [21]:
for trace, trace_file in traces_to_plot.items():
    with open(trace_file) as f:
        j = json.load(f)
        df = pd.DataFrame(j["result"]["responses"])

    single_requests = len(df.loc[df["llmcalls"] == 1])
    single_request_df = df.loc[df["llmcalls"] == 1]
    print(f"Single request interactions: {single_requests}")
    
    total_interactions = len(df.loc[(df["llmcalls"] > 1) & (df["llmcalls"] == df["llmcalls_made"])])
    print(f"Total interactions sent in {trace}: {total_interactions} from {df.loc[(df["llmcalls"] > 1) & (df["llmcalls"] == df["llmcalls_made"])]["adapter_dir"].nunique()} users")
    # remove throttled requests
    df = df.loc[df["request_latency"] >= 0]
    # remove requests finished after cut off
    df["FinishTime"] = df["req_time"] + df["request_latency"]
    df = df.loc[df["FinishTime"] <= cutoff_time]
    # remove single call interactions
    df = df.loc[df["llmcalls"] > 1]
    # get last request of finished interactions
    df = df.loc[df["llmcalls"] == df["llmcalls_made"]]
    # count finished interactions
    finished_interactions = len(df)
    print(f"Number of finished interactions: {finished_interactions} with total requests {df["llmcalls"].sum()}")
    # number of users with finished interactions
    users_finished_interaction = len(df["adapter_dir"].value_counts())
    print(f"Number of user with finished interactions: {users_finished_interaction}")

    users_finished_interaction = len(df["adapter_dir"].value_counts().)
    print(f"Number of user with finished interactions: {users_finished_interaction}")


    # number of single requests served
    single_request_df_served = single_request_df.loc[single_request_df["request_latency"] >= 0]
    single_request_df_served = single_request_df_served.loc[single_request_df_served["req_time"] + single_request_df_served["request_latency"] <= cutoff_time]
    print(f"Number of single requests served: {len(single_request_df_served)}")

Single request interactions: 3179
Total interactions sent in RPM: 1447 from 816 users
Number of finished interactions: 300 with total requests 1080
Number of user with finished interactions: 227
Number of single requests served: 612
Single request interactions: 3179
Total interactions sent in VTC: 1447 from 816 users
Number of finished interactions: 279 with total requests 970
Number of user with finished interactions: 201
Number of single requests served: 800
Single request interactions: 3179
Total interactions sent in FairServe: 1447 from 816 users
Number of finished interactions: 282 with total requests 991
Number of user with finished interactions: 203
Number of single requests served: 805
Single request interactions: 3179
Total interactions sent in FairServe (WSC): 1447 from 816 users
Number of finished interactions: 280 with total requests 985
Number of user with finished interactions: 202
Number of single requests served: 808


In [18]:
with open("../downsampled_data_10000_rows_first_arrivals.csv___vtc_fair___2000___avg.jsonl") as f:
        j = json.load(f)
        df = pd.DataFrame(j["result"]["responses"])
for interaction in df["interaction_id"].unique():
    df_here = df.loc[df["interaction_id"] == interaction]
    plt.plot(df_here["req_time"], df_here["llmcalls_made"])
plt.savefig("tmp.pdf")

In [22]:
with open("../downsampled_data_10000_rows_first_arrivals.csv___vtc_fair___2000___avg.jsonl") as f:
        j = json.load(f)
        df = pd.DataFrame(j["result"]["responses"])
print(len(df))

9241


In [23]:
9241 - 3179

6062